In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from geopy.distance import geodesic
import itertools

# Données entreprises (mockup Ben Arous)
entreprises = pd.DataFrame([
 {"id": 1, "nom": "Acier Ben Arous", "latitude": 36.7353, "longitude": 10.2271,
     "secteur": "sidérurgie", "taille_m2": 25000,
     "flux_sortants": ["chaleur", "scories", "CO2"], "flux_entrants": ["eau", "matières premières", "solvants"]},

    {"id": 2, "nom": "Textile Med", "latitude": 36.7385, "longitude": 10.2596,
     "secteur": "textile", "taille_m2": 8000,
     "flux_sortants": ["eaux usées", "tissus usés", "chaleur"], "flux_entrants": ["chaleur", "eau", "CO2"]},

    {"id": 3, "nom": "PharmaTunis", "latitude": 36.6989, "longitude": 10.1822,
     "secteur": "pharma", "taille_m2": 12000,
     "flux_sortants": ["solvants", "CO2", "chaleur"], "flux_entrants": ["chaleur", "matières premières", "eaux usées"]},

    {"id": 4, "nom": "ChimieSud", "latitude": 36.7682, "longitude": 10.2138,
     "secteur": "chimie", "taille_m2": 15000,
     "flux_sortants": ["vapeur", "CO2", "solvants"], "flux_entrants": ["eau", "solvants", "chaleur"]},

    {"id": 5, "nom": "AgroIndus", "latitude": 36.7261, "longitude": 10.3155,
     "secteur": "agroalimentaire", "taille_m2": 18000,
     "flux_sortants": ["eaux usées", "déchets organiques", "chaleur"], "flux_entrants": ["chaleur", "eau", "CO2"]},
    {"id": 6, "nom": "PlastIndustrie", "latitude": 36.6701, "longitude": 10.2209,
     "secteur": "plasturgie", "taille_m2": 10000,
     "flux_sortants": ["déchets plastiques"], "flux_entrants": ["chaleur", "matières premières"]},
    {"id": 7, "nom": "PapierTunisie", "latitude": 36.7688, "longitude": 10.2756,
     "secteur": "papier", "taille_m2": 9500,
     "flux_sortants": ["chaleur", "eaux usées"], "flux_entrants": ["eau", "bois"]},
    {"id": 8, "nom": "VerreNord", "latitude": 36.7114, "longitude": 10.1578,
     "secteur": "verrerie", "taille_m2": 16000,
     "flux_sortants": ["chaleur", "poussières fines"], "flux_entrants": ["matières premières", "énergie"]},
    {"id": 9, "nom": "Ecopack", "latitude": 36.7293, "longitude": 10.1843,
     "secteur": "emballage", "taille_m2": 11000,
     "flux_sortants": ["déchets carton"], "flux_entrants": ["papier", "chaleur"]},
    {"id": 10, "nom": "Biotex", "latitude": 36.7007, "longitude": 10.1521,
     "secteur": "textile", "taille_m2": 9000,
     "flux_sortants": ["tissus usés", "eaux usées"], "flux_entrants": ["eau", "matières premières"]}
])

# Génération des couples
paires = []
for a, b in itertools.permutations(entreprises.to_dict(orient="records"), 2):
    dist = geodesic((a["latitude"], a["longitude"]), (b["latitude"], b["longitude"])).km
    match_flux = set(a["flux_sortants"]) & set(b["flux_entrants"])
    match_flux_back = set(b["flux_sortants"]) & set(a["flux_entrants"])
    bidirectionnelle = len(match_flux) > 0 and len(match_flux_back) > 0
    synergie = int((len(match_flux) > 0 or len(match_flux_back) > 0) and dist < 10)

    paires.append({
        "A_id": a["id"], "B_id": b["id"],
        "distance_km": dist,
        "flux_A_vers_B": list(match_flux),
        "flux_B_vers_A": list(match_flux_back),
        "bidirectionnelle": bidirectionnelle,
        "synergie_possible": synergie
    })

df_paires = pd.DataFrame(paires)

# Apprentissage simple par IA (logistic regression)
X = df_paires[["distance_km", "bidirectionnelle"]].copy()
X["flux_match"] = df_paires["flux_A_vers_B"].apply(lambda x: len(x))
X["flux_match_back"] = df_paires["flux_B_vers_A"].apply(lambda x: len(x))
y = df_paires["synergie_possible"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model = LogisticRegression()
model.fit(X_train, y_train)

df_paires["synergie_predite"] = model.predict(X)

# Nettoyage : on garde que les paires avec synergie prédite = 1 et sans doublons (A < B)
synergies = df_paires[df_paires["synergie_predite"] == 1]
synergies = synergies[synergies["A_id"] < synergies["B_id"]]

# Fusion noms entreprises
synergies = synergies.merge(entreprises[["id", "nom"]], left_on="A_id", right_on="id", how="left").rename(columns={"nom": "Entreprise_A"})
synergies = synergies.merge(entreprises[["id", "nom"]], left_on="B_id", right_on="id", how="left").rename(columns={"nom": "Entreprise_B"})

# Affichage clair
print("\n=== 🔍 SYNERGIES INDUSTRIELLES DÉTECTÉES PAR L'IA ===")
for _, row in synergies.sort_values(by="distance_km").iterrows():
    flux_list = row["flux_A_vers_B"] + row["flux_B_vers_A"]
    flux_str = ", ".join(set(flux_list))
    bidir_str = "✅ Oui" if row["bidirectionnelle"] else "❌ Non"
    print(f"🔹 {row['Entreprise_A']} ↔ {row['Entreprise_B']} | 📍 Distance : {row['distance_km']:.2f} km | ♻️ Flux : {flux_str} | 🔁 Bidirectionnelle : {bidir_str}")



=== 🔍 SYNERGIES INDUSTRIELLES DÉTECTÉES PAR L'IA ===
🔹 VerreNord ↔ Biotex | 📍 Distance : 1.29 km | ♻️ Flux :  | 🔁 Bidirectionnelle : ❌ Non
🔹 PharmaTunis ↔ VerreNord | 📍 Distance : 2.58 km | ♻️ Flux : chaleur | 🔁 Bidirectionnelle : ❌ Non
🔹 PharmaTunis ↔ Biotex | 📍 Distance : 2.70 km | ♻️ Flux : eaux usées | 🔁 Bidirectionnelle : ❌ Non
🔹 Acier Ben Arous ↔ Textile Med | 📍 Distance : 2.92 km | ♻️ Flux : CO2, chaleur | 🔁 Bidirectionnelle : ❌ Non
🔹 VerreNord ↔ Ecopack | 📍 Distance : 3.09 km | ♻️ Flux : chaleur | 🔁 Bidirectionnelle : ❌ Non
🔹 PharmaTunis ↔ Ecopack | 📍 Distance : 3.38 km | ♻️ Flux : chaleur | 🔁 Bidirectionnelle : ❌ Non
🔹 Textile Med ↔ PapierTunisie | 📍 Distance : 3.65 km | ♻️ Flux : chaleur | 🔁 Bidirectionnelle : ❌ Non
🔹 Acier Ben Arous ↔ ChimieSud | 📍 Distance : 3.84 km | ♻️ Flux : chaleur, solvants | 🔁 Bidirectionnelle : ✅ Oui
🔹 Acier Ben Arous ↔ Ecopack | 📍 Distance : 3.88 km | ♻️ Flux : chaleur | 🔁 Bidirectionnelle : ❌ Non
🔹 Ecopack ↔ Biotex | 📍 Distance : 4.28 km | ♻️ Flux